In [30]:
import requests
import requests_cache
from matplotlib import pyplot as plt
import lxml.html
import pandas as pd
import re
import json 
import lxml.html as lx
import urllib
from urllib2 import Request, urlopen, HTTPError
from urlparse import urlunparse, urlparse, urljoin
import numpy 
import plotly.plotly as py
import plotly.figure_factory as ff
import math
from bs4 import BeautifulSoup
import plotly.graph_objs as go 
import plotly
import plotly.tools as tls
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import Scatter, Figure, Layout
init_notebook_mode(connected=True)
import cufflinks as cf

from datetime import datetime
import pandas_datareader as web
from plotly.graph_objs import *

In [4]:
def fetch_page(url):
    response = requests.get(url)
    response.raise_for_status()
    return response

url="https://www.zillow.com/browse/homes/ca/"
requests_cache.install_cache("cache")
page = fetch_page(url)
html = lx.fromstring(page.text)
county_html = html.xpath('//main/div/div/ul/li/a')
county = [i.text_content() for i in county_html]
county_name = pd.DataFrame({"RegionName":[" ".join(k) for k in [i.split(" ")[:-2] for i in county]]})

In [5]:
#Import Housing Data
import os.path

def dataframe(i = 0):
    
    """Read datasets
    Please change the file path to fit to your computer
    Input:the name of dataset //dataname   
    """
    
    file_path = "/Users/HUI/Dropbox/16_UCDavis/17W_STA141B/05_Project/Housing_Data"
    file_name = ["County_MedianSoldPrice_AllHomes.csv","County_MedianListingPrice_AllHomes.csv","County_MedianListingPrice_1Bedroom.csv",
                 "County_MedianListingPrice_2Bedroom.csv","County_MedianListingPrice_3Bedroom.csv","County_MedianListingPrice_4Bedroom.csv",
                 "County_MedianListingPrice_5BedroomOrMore.csv","County_MedianListingPrice_CondoCoop.csv","County_MedianListingPrice_DuplexTriplex.csv",
                 "County_MedianListingPrice_Sfr.csv"]
    dataframe = pd.read_csv(os.path.join(file_path,file_name[i]),header=0)
    CA_dataframe = dataframe[dataframe["State"]== "CA"]
    CA_dataframe = CA_dataframe.reset_index(drop=True)
    return(CA_dataframe)

###name of eachdata
dataname=["sold","list","one","two","thre","four","five","cond","trip","sfr"]

###save all of dataset to CA dict.
CA = {i: dataframe(dataname.index(i)) for i in dataname}

In [6]:
#All Home Sold time-series data 1996-2016
CA["sold"] = pd.merge(pd.DataFrame(county_name),CA["sold"], how='left', on=['RegionName'])
CA_sold_t = CA["sold"].transpose()
CA_sold_t = CA_sold_t.rename(columns=lambda i: CA["sold"]["RegionName"][i])[7:]
date = pd.date_range('1996-04', '2016-07', freq='M')
CA_sold_t['date'] = date

#sold price time-series data
CA_sold_t = CA_sold_t.set_index('date')
CA_sold_t.head()

,Los Angeles,Orange,San Diego,Riverside,San Bernardino,Santa Clara,Alameda,Sacramento,Contra Costa,Fresno,...,Glenn,Plumas,Colusa,Mariposa,Inyo,Trinity,Mono,Modoc,Sierra,Alpine
date,,,,,,,,,,,,,,,,,,,,,
1996-04-30,162225,191525,158225,114550,100250,234325,196275,120425,192750,94850,...,67000,NaN,NaN,NaN,121471,53000,77500,NaN,NaN,NaN
1996-05-31,162925,197550,158125,112500,101700,238575,196900,121675,203000,95475,...,65823.5,NaN,97230.8,115000,NaN,NaN,78970.6,NaN,NaN,NaN
1996-06-30,164500,197875,158925,115000,103450,238875,202225,122175,209050,95475,...,NaN,NaN,NaN,NaN,119077,51950,NaN,NaN,NaN,NaN
1996-07-31,164550,198050,158700,113975,101850,241875,203700,122175,207150,96550,...,NaN,NaN,NaN,94230.8,112882,NaN,81538.5,NaN,NaN,NaN
1996-08-31,163300,199300,157300,113225,101650,241200,202250,121075,202500,96625,...,NaN,80000,NaN,NaN,NaN,82300,98470.6,NaN,NaN,NaN


In [7]:
#Different type of house average yr 2015/2016
def aver(i="one"):
    data = CA[i]
    CA_2015 = data.iloc[:,[k.__contains__("2015") for k in data.keys()]]
    CA_2016 = data.iloc[:,[k.__contains__("2016") for k in data.keys()]]
    CA_mean = pd.DataFrame({i+"_15":CA_2015.mean(1),i+"_16":CA_2016.mean(1)})
    CA_ave_name = pd.DataFrame({"RegionName":data["RegionName"]})
    CA_ave = pd.concat([CA_ave_name,CA_mean],axis=1)
    return CA_ave
CA_ave=[aver(i) for i in dataname]


#average price in yr 25/26 for different type of house/and all homes
diff_ave = reduce(lambda left,right: pd.merge(left,right,how="left",on='RegionName'), CA_ave)
diff_ave.head()

,RegionName,sold_15,sold_16,list_15,list_16,one_15,one_16,two_15,two_16,thre_15,...,four_15,four_16,five_15,five_16,cond_15,cond_16,trip_15,trip_16,sfr_15,sfr_16
0,Los Angeles,498737.500000,518508.333333,537666.666667,587328.666667,363468.333333,409424.500000,433116.583333,469545.750000,497107.916667,...,659808.333333,701394.000000,1.038750e+06,1.176233e+06,452570.833333,488478.750000,664705.500000,7.312750e+05,576524.500000,629662.333333
1,Orange,601020.833333,623150.000000,675932.083333,705890.000000,279250.000000,303945.833333,417687.500000,441220.833333,639658.333333,...,841700.000000,883811.583333,1.382648e+06,1.439552e+06,422483.333333,446204.166667,924108.583333,1.120979e+06,787609.750000,820343.541667
2,San Diego,472187.500000,489250.000000,538650.000000,575525.000000,276475.000000,319328.625000,360778.166667,396704.083333,494828.166667,...,684390.666667,716899.958333,1.093432e+06,1.184135e+06,376438.583333,410828.333333,678408.333333,7.657750e+05,609515.250000,641470.583333
3,Riverside,309068.750000,320437.500000,344257.500000,361732.458333,145399.875000,145940.750000,227945.750000,237962.041667,327233.333333,...,394170.750000,413642.833333,4.613150e+05,4.925966e+05,273656.666667,280891.666667,352866.583333,3.872125e+05,356255.250000,376051.708333
4,San Bernardino,264327.083333,272666.666667,271312.500000,291733.333333,NaN,NaN,172779.083333,185468.166667,255624.833333,...,367520.833333,385991.166667,4.659978e+05,4.931395e+05,259899.916667,271735.416667,295638.125000,3.068167e+05,272363.916667,293408.333333


In [8]:
file_path = "/Users/HUI/Dropbox/16_UCDavis/17W_STA141B/05_Project/"
def readdata(file_name):
             #,mergedata=diff_ave):

    dataframe = pd.read_csv(os.path.join(file_path,file_name),header=0)
    
    return(dataframe)

Unemp_frame = readdata("Unemploy_Rate.csv")
Unemp_frame["County"] = [" ".join(i.split(" ")[:-1]) for i in Unemp_frame["County"]]
Unemp_frame.head()

,County,Unemploy_Rate
0,Alameda,4.3
1,Alpine,5.9
2,Amador,6.3
3,Butte,6.9
4,Calaveras,6.2


In [9]:
###ZipCode housing price
def dataframe_zip(i = 0):
    
    """Read datasets
    Please change the file path to fit to your computer
    Input:the name of dataset //dataname   
    """
    
    file_path = "/Users/HUI/Dropbox/16_UCDavis/17W_STA141B/05_Project/Housing_Data"
    file_name = ["Zip_MedianSoldPrice_AllHomes.csv","Zip_MedianListingPrice_AllHomes.csv","Zip_MedianListingPrice_1Bedroom.csv",
                 "Zip_MedianListingPrice_2Bedroom.csv","Zip_MedianListingPrice_3Bedroom.csv","Zip_MedianListingPrice_4Bedroom.csv",
                 "Zip_MedianListingPrice_5BedroomOrMore.csv","Zip_MedianListingPrice_CondoCoop.csv","Zip_MedianListingPrice_DuplexTriplex.csv",
                 "Zip_MedianListingPrice_Sfr.csv"]
    dataframe = pd.read_csv(os.path.join(file_path,file_name[i]),header=0)
    CA_dataframe = dataframe[dataframe["State"]== "CA"]
    CA_dataframe = CA_dataframe.reset_index(drop=True)
    return(CA_dataframe)

###name of eachdata
dataname=["sold","list","one","two","thre","four","five","cond","trip","sfr"]

###save all of dataset to CA dict.
CA_zip = {i: dataframe_zip(dataname.index(i)) for i in dataname}
CA_zip['sold'].head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06
0,97564,94109,San Francisco,CA,San Francisco,San Francisco,8,259250.0,252000.0,263950.0,...,1123750.0,1080500.0,969350.0,1070900.0,NaN,NaN,8.990000e+05,1.017235e+06,979850.0,NaN
1,96107,90250,Hawthorne,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,16,155850.0,157750.0,165725.0,...,510550.0,487775.0,NaN,529000.0,485294.1176,441000.0,4.716500e+05,5.026000e+05,501500.0,514500.0
2,96027,90046,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,26,288825.0,285925.0,309250.0,...,1020750.0,1017500.0,875750.0,824650.0,767475.0000,NaN,1.399450e+06,1.193000e+06,NaN,979100.0
3,97771,94565,Pittsburg,CA,San Francisco,Contra Costa,27,123800.0,125750.0,119225.0,...,327250.0,326800.0,327600.0,339625.0,322225.0000,325000.0,3.372500e+05,3.502500e+05,358000.0,355125.0
4,97711,94501,Alameda,CA,San Francisco,Alameda,50,191650.0,193950.0,193700.0,...,742100.0,754000.0,766650.0,790750.0,746500.0000,NaN,8.226923e+05,8.245294e+05,796300.0,753600.0


In [10]:
#Different type of house average yr 2015/2016 for zip code
def aver_zip(i="one"):
    data = CA_zip[i]
    CA_2015 = data.iloc[:,[k.__contains__("2015") for k in data.keys()]]
    CA_2016 = data.iloc[:,[k.__contains__("2016") for k in data.keys()]]
    CA_mean = pd.DataFrame({i+"_15":CA_2015.mean(1),i+"_16":CA_2016.mean(1)})
    CA_ave_name = pd.DataFrame({"RegionName":data["RegionName"],"CountyName":data["CountyName"]})
    CA_ave_zip = pd.concat([CA_ave_name,CA_mean],axis=1)
    return CA_ave_zip

CA_ave_zip=[aver_zip(i) for i in dataname]


#average price in yr 25/26 for different type of house/and all homes
diff_ave_zip = reduce(lambda left,right:pd.merge(left, right,how='left', on=['RegionName','CountyName']) , CA_ave_zip)
diff_ave_zip.head()

,CountyName,RegionName,sold_15,sold_16,list_15,list_16,one_15,one_16,two_15,two_16,...,four_15,four_16,five_15,five_16,cond_15,cond_16,trip_15,trip_16,sfr_15,sfr_16
0,San Francisco,94109,994933.333333,9.653618e+05,1.155800e+06,1.096792e+06,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1195000.0,1.046583e+06,NaN,NaN,NaN,NaN
1,Los Angeles,90250,522645.825800,4.860907e+05,4.796205e+05,5.443083e+05,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.780625e+05,5.364279e+05
2,Los Angeles,90046,936914.819010,1.084756e+06,1.220125e+06,1.371871e+06,NaN,NaN,NaN,NaN,...,2.235417e+06,2.284125e+06,NaN,NaN,NaN,6.584330e+05,NaN,NaN,1.623625e+06,1.814831e+06
3,Contra Costa,94565,318502.083333,3.413083e+05,3.400394e+05,3.852135e+05,NaN,NaN,NaN,NaN,...,3.879000e+05,4.395845e+05,NaN,NaN,NaN,NaN,NaN,NaN,3.434932e+05,3.869069e+05
4,Alameda,94501,745666.666667,7.887243e+05,NaN,8.449757e+05,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.029506e+05,8.911818e+05


In [11]:
#conbine zipcode data with school rank#
file_path = "/Users/HUI/Dropbox/16_UCDavis/17W_STA141B/05_Project"
school = pd.read_excel(os.path.join(file_path,"school.xlsx"),skiprows = [1,2,3], header = 1)
school.head()
school_rank = school.iloc[:,[0,1,3,5,6,13,14,15,17]]
school_rank.columns = ['Rank2016','District','City','RegionName','County','Score2016','Score2015','Rank2015','SchoolDiggerRating']
school_rank
zip_school_home= pd.merge(school_rank,diff_ave_zip,on="RegionName")
zip_school_home

zip_school_home.to_csv("/Users/HUI/Dropbox/16_UCDavis/17W_STA141B/05_Project/zip_school_price.csv")


In [12]:
Pop_frame = readdata("population.csv")
Pop_frame = Pop_frame[["County","Population"]]
Pop_frame.head()

,County,Population
0,Alameda,1638215
1,Alpine,1110
2,Amador,37001
3,Butte,225411
4,Calaveras,44828


In [13]:
Unem_diff=pd.merge(diff_ave,Unemp_frame,how="left",left_on = "RegionName",right_on = "County")
Unem_diff.head()

,RegionName,sold_15,sold_16,list_15,list_16,one_15,one_16,two_15,two_16,thre_15,...,five_15,five_16,cond_15,cond_16,trip_15,trip_16,sfr_15,sfr_16,County,Unemploy_Rate
0,Los Angeles,498737.500000,518508.333333,537666.666667,587328.666667,363468.333333,409424.500000,433116.583333,469545.750000,497107.916667,...,1.038750e+06,1.176233e+06,452570.833333,488478.750000,664705.500000,7.312750e+05,576524.500000,629662.333333,Los Angeles,5.7
1,Orange,601020.833333,623150.000000,675932.083333,705890.000000,279250.000000,303945.833333,417687.500000,441220.833333,639658.333333,...,1.382648e+06,1.439552e+06,422483.333333,446204.166667,924108.583333,1.120979e+06,787609.750000,820343.541667,Orange,4.1
2,San Diego,472187.500000,489250.000000,538650.000000,575525.000000,276475.000000,319328.625000,360778.166667,396704.083333,494828.166667,...,1.093432e+06,1.184135e+06,376438.583333,410828.333333,678408.333333,7.657750e+05,609515.250000,641470.583333,San Diego,4.8
3,Riverside,309068.750000,320437.500000,344257.500000,361732.458333,145399.875000,145940.750000,227945.750000,237962.041667,327233.333333,...,4.613150e+05,4.925966e+05,273656.666667,280891.666667,352866.583333,3.872125e+05,356255.250000,376051.708333,Riverside,6.0
4,San Bernardino,264327.083333,272666.666667,271312.500000,291733.333333,NaN,NaN,172779.083333,185468.166667,255624.833333,...,4.659978e+05,4.931395e+05,259899.916667,271735.416667,295638.125000,3.068167e+05,272363.916667,293408.333333,San Bernardino,5.7


In [14]:
Unem_diff_pop=pd.merge(Unem_diff,Pop_frame,how="left",left_on = "RegionName",right_on = "County")
Unem_diff_pop.head()

,RegionName,sold_15,sold_16,list_15,list_16,one_15,one_16,two_15,two_16,thre_15,...,cond_15,cond_16,trip_15,trip_16,sfr_15,sfr_16,County_x,Unemploy_Rate,County_y,Population
0,Los Angeles,498737.500000,518508.333333,537666.666667,587328.666667,363468.333333,409424.500000,433116.583333,469545.750000,497107.916667,...,452570.833333,488478.750000,664705.500000,7.312750e+05,576524.500000,629662.333333,Los Angeles,5.7,Los Angeles,10170292
1,Orange,601020.833333,623150.000000,675932.083333,705890.000000,279250.000000,303945.833333,417687.500000,441220.833333,639658.333333,...,422483.333333,446204.166667,924108.583333,1.120979e+06,787609.750000,820343.541667,Orange,4.1,Orange,3169776
2,San Diego,472187.500000,489250.000000,538650.000000,575525.000000,276475.000000,319328.625000,360778.166667,396704.083333,494828.166667,...,376438.583333,410828.333333,678408.333333,7.657750e+05,609515.250000,641470.583333,San Diego,4.8,San Diego,3299521
3,Riverside,309068.750000,320437.500000,344257.500000,361732.458333,145399.875000,145940.750000,227945.750000,237962.041667,327233.333333,...,273656.666667,280891.666667,352866.583333,3.872125e+05,356255.250000,376051.708333,Riverside,6.0,Riverside,2361026
4,San Bernardino,264327.083333,272666.666667,271312.500000,291733.333333,NaN,NaN,172779.083333,185468.166667,255624.833333,...,259899.916667,271735.416667,295638.125000,3.068167e+05,272363.916667,293408.333333,San Bernardino,5.7,San Bernardino,2128133


In [15]:
crim = readdata("crime.csv")
Unme_diff_pop_crim = pd.merge(Unem_diff_pop,crim.iloc[:,1:],how="left", on = "RegionName")
Unme_diff_pop_crim.head()

,RegionName,sold_15,sold_16,list_15,list_16,one_15,one_16,two_15,two_16,thre_15,...,cond_16,trip_15,trip_16,sfr_15,sfr_16,County_x,Unemploy_Rate,County_y,Population,CrimeCases
0,Los Angeles,498737.500000,518508.333333,537666.666667,587328.666667,363468.333333,409424.500000,433116.583333,469545.750000,497107.916667,...,488478.750000,664705.500000,7.312750e+05,576524.500000,629662.333333,Los Angeles,5.7,Los Angeles,10170292,43163.0
1,Orange,601020.833333,623150.000000,675932.083333,705890.000000,279250.000000,303945.833333,417687.500000,441220.833333,639658.333333,...,446204.166667,924108.583333,1.120979e+06,787609.750000,820343.541667,Orange,4.1,Orange,3169776,2967.0
2,San Diego,472187.500000,489250.000000,538650.000000,575525.000000,276475.000000,319328.625000,360778.166667,396704.083333,494828.166667,...,410828.333333,678408.333333,7.657750e+05,609515.250000,641470.583333,San Diego,4.8,San Diego,3299521,11931.0
3,Riverside,309068.750000,320437.500000,344257.500000,361732.458333,145399.875000,145940.750000,227945.750000,237962.041667,327233.333333,...,280891.666667,352866.583333,3.872125e+05,356255.250000,376051.708333,Riverside,6.0,Riverside,2361026,17148.0
4,San Bernardino,264327.083333,272666.666667,271312.500000,291733.333333,NaN,NaN,172779.083333,185468.166667,255624.833333,...,271735.416667,295638.125000,3.068167e+05,272363.916667,293408.333333,San Bernardino,5.7,San Bernardino,2128133,12956.0


In [16]:
Unme_diff_pop_crim.keys()

Index([u'RegionName', u'sold_15', u'sold_16', u'list_15', u'list_16',
       u'one_15', u'one_16', u'two_15', u'two_16', u'thre_15', u'thre_16',
       u'four_15', u'four_16', u'five_15', u'five_16', u'cond_15', u'cond_16',
       u'trip_15', u'trip_16', u'sfr_15', u'sfr_16', u'County_x',
       u'Unemploy_Rate', u'County_y', u'Population', u'CrimeCases'],
      dtype='object')

In [17]:
Unme_diff_pop_crim.drop(["County_x",'County_y'],axis=1, inplace=True)

In [18]:
rank = readdata("county_all_rank.csv")
all_rank = pd.merge(Unme_diff_pop_crim,rank,how="left", left_on = "RegionName",right_on = "County")
all_rank.drop(['County'],axis=1, inplace=True)

In [19]:
income = readdata("income.csv")
#income = income.drop(income.iloc[:,:3])
all_income = pd.merge(all_rank,income.iloc[:,3:],how="left", left_on = "RegionName",right_on = "county")
all_income.drop(['county'],axis=1, inplace=True)

In [20]:
CA_sold_t.to_csv("/Users/HUI/Dropbox/16_UCDavis/17W_STA141B/05_Project/CA_sold_t.csv")
diff_ave.to_csv("/Users/HUI/Dropbox/16_UCDavis/17W_STA141B/05_Project/diff_ave.csv")

In [21]:
all_income.to_csv("/Users/HUI/Dropbox/16_UCDavis/17W_STA141B/05_Project/all_data.csv")

In [22]:
all_data = all_income

In [65]:
####plot multiple time-series line###

def trace(x,y,name):
    trace = Scatter(
            x=x,y=y,
            line=Line(
                width=3
            ),
            name=name
        )
    return trace
traceall = list()

for i in CA_sold_t.keys():
    traceall.append(trace(x=CA_sold_t.index,y=CA_sold_t[i],name=i))


TF = (numpy.identity(len(CA_sold_t.keys()))==1).tolist()

def buttons(TF, label):
    buttons = dict(
        args=['visible',TF],
        label = label,
        method='restyle'
    )
    return buttons

buttonsall = list()
for i in range(len(CA_sold_t.keys())):
    buttonsall.append(buttons(TF=TF[i],label=CA_sold_t.keys()[i]))
    
layout = Layout(
    title='Median Sold Price from 1996 to 2016 in Califorina county',
    updatemenus=list([
        dict(
            x=-0.05,
            y=1,
            yanchor='top',
            buttons=buttonsall,
        )
    ]),
)

fig = Figure(data=traceall, layout=layout)
py.iplot(fig)

In [25]:
####plot multiple time-series line###

upper_bound0 = go.Scatter(
    name='Upper Bound_med',
    x=CA_sold_t.index,
    y=CA_sold_t.mean(axis=1)+CA_sold_t.std(axis=1),
    mode='lines',
    marker=dict(color="444"),
    line=dict(width=0),
    fillcolor='rgba(68, 68, 68, 0.3)',
    fill='tonexty' )

trace0 = go.Scatter(
    x=CA_sold_t.index,
    y=CA_sold_t.median(axis=1),
    fillcolor='rgba(68, 68, 68, 0.3)',
    name = "median",
    fill='tonexty'
)

trace1 = go.Scatter(
    x=CA_sold_t.index,
    y=CA_sold_t.mean(axis=1),
    line=dict(color='rgb(31, 119, 180)'),
    fillcolor='rgba(68, 68, 68, 0.3)',
    name = "mean",
    fill='tonexty'
)

lower_bound0 = go.Scatter(
    name='Lower Bound_med',
    x=CA_sold_t.index,
    y=CA_sold_t.mean(axis=1)-CA_sold_t.std(axis=1),
    marker=dict(color="444"),
    line=dict(width=0),
    mode='lines' )


data = [lower_bound0,trace0,trace1,upper_bound0]



layout = dict(
    title='Sold Price from 1996 to 2016 in Califorina',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=6,
                     label='6m',
                     step='month',
                     stepmode='backward'),
                dict(count=1,
                    label='1y',
                    step='year',
                    stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(),
        type='date'
    )
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

In [28]:
###medain sales price compare with US and CA in 2015

median_us=296400
all_data["sold_15"].median()


trace0 = go.Bar(
    x=[median_us, all_data["sold_15"].mean()],
    y=['US ','California'],
    orientation = 'h',
    marker=dict(
        color=['rgba(204,204,204,1)','rgba(222,45,38,0.8)']),
)

data = [trace0]

layout = go.Layout(
    title='Medidan Housing Price between US and California',
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='color-bar')


In [76]:
#url = 'http://catalog.civicdashboards.com/dataset/ce409ee1-5128-4b66-aa3e-957dbd4de8ba/resource/6f805645-0836-478d-b168-c1f72d53b4f3/download/5faf934922fd4748a843d55990297d9ftemp.geojson'
california_data = json.load(open('/Users/HUI/Dropbox/16_UCDavis/17W_STA141B/05_Project/california.json'))

In [77]:
import urllib
import ast
import HTMLParser
import string
import folium
import json
from branca.utilities import _locations_mirror
from folium.features import *
from folium.features import _locations_tolist

In [78]:
class MultiPolygon(MacroElement):
    """
    !! This is hacked from folium.features.MultiPolyLine !! 
    
    """
    def __init__(self, locations, color=None, weight=None,
                 opacity=None, latlon=True, popup=None):
        super(MultiPolygon, self).__init__()
        self._name = 'MultiPolygon'
        self.data = (_locations_mirror(locations) if not latlon else
                     _locations_tolist(locations))
        self.color = color
        self.weight = weight
        self.opacity = opacity
        if isinstance(popup, text_type) or isinstance(popup, binary_type):
            self.add_children(Popup(popup))
        elif popup is not None:
            self.add_children(popup)

        self._template = Template(u"""
            {% macro script(this, kwargs) %}
                var {{this.get_name()}} = L.multiPolygon(
                    {{this.data}},
                    {
                        {% if this.color != None %}color: '{{ this.color }}',{% endif %}
                        {% if this.weight != None %}weight: {{ this.weight }},{% endif %}
                        {% if this.opacity != None %}opacity: {{ this.opacity }},{% endif %}
                        });
                {{this._parent.get_name()}}.addLayer({{this.get_name()}});
            {% endmacro %}
            """)  # noqa

    def _get_self_bounds(self):
        """Computes the bounds of the object itself (not including it's children)
        in the form [[lat_min, lon_min], [lat_max, lon_max]]
        """
        bounds = [[None, None], [None, None]]
        for point in iter_points(self.data):
            bounds = [
                [
                    none_min(bounds[0][0], point[0]),
                    none_min(bounds[0][1], point[1]),
                ],
                [
                    none_max(bounds[1][0], point[0]),
                    none_max(bounds[1][1], point[1]),
                ],
            ]
        return bounds

In [79]:
geo_dict = {}
for x in range(len(california_data['features'])):
    # I ignore the last eleven characters in the name since the geojson file includes ' County, CO' in the county names and the population data does not
    name = california_data['features'][x]['properties']['name'][:-11]
    if name in all_data['RegionName'].unique():
        geo_dict[name] = california_data['features'][x]
    else:
        print 'not in: ', name

ser = pd.Series(geo_dict.values(), index = geo_dict.keys())
ser.name = 'coordinates'
all_data_cord = all_data.join(ser, on='RegionName')

income_rate = [round(i,6) for i in all_data["total_income"]/all_data["Population"]]
crim_rate = [round(i,6) for i in all_data["CrimeCases"]/all_data["Population"]]

all_data_sub = all_data_cord[['RegionName','sold_15','sold_16','list_15','list_16','Score2016','Score2015','Unemploy_Rate','Population',"coordinates"]]
all_data_sub['income_rate'] = income_rate
all_data_sub['crim_rate'] = crim_rate


/Users/HUI/miniconda2/lib/python2.7/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/HUI/miniconda2/lib/python2.7/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [80]:
colors = ['#ffffe0','#fffddb','#fffad7','#fff7d1','#fff5cd','#fff2c8',
          '#fff0c4','#ffedbf','#ffebba','#ffe9b7','#ffe5b2','#ffe3af',
          '#ffe0ab','#ffdda7','#ffdba4','#ffd9a0','#ffd69c','#ffd399',
          '#ffd196','#ffcd93','#ffca90','#ffc88d','#ffc58a','#ffc288',
          '#ffbf86','#ffbd83','#ffb981','#ffb67f','#ffb47d','#ffb17b',
          '#ffad79','#ffaa77','#ffa775','#ffa474','#ffa172','#ff9e70',
          '#ff9b6f','#ff986e','#ff956c','#fe916b','#fe8f6a','#fd8b69',
          '#fc8868','#fb8567','#fa8266','#f98065','#f87d64','#f77a63',
          '#f67862','#f57562','#f37261','#f37060','#f16c5f','#f0695e',
          '#ee665d','#ed645c','#ec615b','#ea5e5b','#e85b59','#e75859',
          '#e55658','#e45356','#e35056','#e14d54','#df4a53','#dd4852',
          '#db4551','#d9434f','#d8404e','#d53d4d','#d43b4b','#d2384a',
          '#cf3548','#cd3346','#cc3045','#ca2e43','#c72b42','#c52940',
          '#c2263d','#c0233c','#be213a','#bb1e37','#ba1c35','#b71933',
          '#b41731','#b2152e','#b0122c','#ac1029','#aa0e27','#a70b24',
          '#a40921','#a2071f','#a0051c','#9d0419','#990215','#970212',
          '#94010e','#91000a','#8e0006','#8b0000', '#8b0000']

scl = dict(zip(range(0, 101), colors))
colorscl = [[i * .01, v] for i,v in enumerate(colors)]

In [103]:
def get_scl(obj,number):
    """
    assign color
    """
    if pd.isnull(obj):
        return scl[0] 
    else:
        frac = int(obj/number)
        return scl[frac]

all_data_sub['color'] = all_data_sub['sold_15'].apply(get_scl,number = 12500)


/Users/HUI/miniconda2/lib/python2.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [96]:
requests_cache.install_cache("cache") 
url = requests.get("https://en.wikipedia.org/wiki/User:Michael_J/County_table") 
soup = BeautifulSoup(url.text)

sort = []
state = []
FIPS = []
county = []
county_seat = []
population = []
land_area_km = []
land_area_ml = []
water_area_km = []
water_area_ml = []
total_area_km = []
total_area_ml = []
lat = []
lon = []
 
table = soup.find("table", { "class" : "wikitable sortable" })
 
for row in table.findAll("tr"):
    cells = row.findAll("td")
    #For each "tr", assign each "td" to a variable.
    if len(cells) == 14:
        sort.append(cells[0].find(text=True))
        state.append(cells[1].find(text=True))
        FIPS.append(cells[2].find(text=True))
        county.append(cells[3].find(text=True))
        county_seat.append(cells[4].find(text=True))
        population.append(cells[5].find(text=True))
        land_area_km.append(cells[6].find(text=True))
        land_area_ml.append(cells[7].find(text=True))
        water_area_km.append(cells[8].find(text=True))
        water_area_ml.append(cells[9].find(text=True))
        total_area_km.append(cells[10].find(text=True))
        total_area_ml.append(cells[11].find(text=True))
        lat.append(cells[12].find(text=True))
        lon.append(cells[13].find(text=True))

cord = [('county', county[186:244]),
         ('lat', lat[186:244]),
         ('lon', lon[186:244]),
         ]
center_cord = pd.DataFrame.from_items(cord)

all_data_sub_cen = pd.merge(all_data_sub, center_cord, left_on='RegionName', right_on='county')
all_data_sub_cen["lon"] = ['-'+i[1:9] for i in all_data_sub_cen["lon"]]
all_data_sub_cen["lat"] = [i[1:8] for i in all_data_sub_cen["lat"]]


In [83]:
mapbox_access_token = 'pk.eyJ1IjoiZGFuaWVsbGU5MTUxNSIsImEiOiJjaXpyeW44cTYwMGtvMzJsOXZtNTEwNmJtIn0.8d-m0sZPWnlRzH-T4OlAiA'

In [84]:
layers_ls = []
for x in all_data_sub.index:
    item_dict = dict(sourcetype = 'geojson',
                     source = all_data_sub.ix[x]['coordinates'],
                     type = 'fill',
                     color = all_data_sub.ix[x]['color'])
    layers_ls.append(item_dict)
test = []
for i in range(len(all_data_sub_cen["RegionName"].tolist())):
    test.append(all_data_sub_cen["RegionName"].tolist()[i] + '</br>' + 
                'Sold Price: ' +str(all_data_sub_cen["sold_15"].tolist()[i])+ '</br>' + 
                'List Price: ' +str(all_data_sub_cen["list_15"].tolist()[i])+ '</br>' + 
                'School Score: '+str(all_data_sub_cen["Score2015"].tolist()[i])+ '</br>' +
                'Population:' +str(all_data_sub_cen["Population"].tolist()[i]) +'</br>' +
                'Ave Income: '+str(all_data_sub_cen["income_rate"].tolist()[i]) +'</br>' +
                'Criminal Rate: '+str(all_data_sub_cen["crim_rate"].tolist()[i])+'</br>' +
                'Unemploy Rate: '+str(all_data_sub_cen["Unemploy_Rate"].tolist()[i]))

In [165]:
data = go.Data([
        go.Scattermapbox(
            lat=all_data_sub_cen["lat"].tolist(),
            lon=all_data_sub_cen["lon"].tolist(),
            text=test,
            mode = 'markers',
                    marker = go.Marker(cmax=int(all_data_sub['sold_15'].max()/0.01),cmin=0,colorscale = colorscl,
                        showscale = True,autocolorscale=False,color=range(0,100),
                    colorbar= go.ColorBar(len = .89)
                                       ),
        )
    ])


layout = go.Layout(
    height=600,
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        layers=layers_ls,
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=39,
            lon=-124
        ),
        pitch=0,
        zoom=5,
        style='light'
               )
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='county-level-choropleths-python')

In [41]:
#plot for scatter matrix
crim_rate = ({"crimerate":all_data["CrimeCases"]/all_data["Population"]})
crim_rate_log = ({"logcrimerate":[math.log(i) for i in all_data["CrimeCases"]/all_data["Population"]]})
income_rate = ({"incomerate":all_data["total_income"]/all_data["Population"]})
population_log =  ({"logpopulation":[math.log(i) for i in all_data["Population"]]})
dataframe = all_data[['sold_15','Unemploy_Rate','Score2015']]

all_data_sub_2 = pd.concat([dataframe,pd.DataFrame(crim_rate_log),
                          pd.DataFrame(population_log),pd.DataFrame(income_rate)],axis=1)

fig = ff.create_scatterplotmatrix(all_data_sub_2, diag='histogram',
                                  colormap='Blues', height=800, width=800)
py.iplot(fig, filename = 'Scatterplot Matrix')

In [102]:
###Recent 5 year growth rate in county
###Calcuat CAGR Compound Annual Growth Rate
CAGR= ({'CAGR':((CA_sold_t['2016'].mean(axis=0)/CA_sold_t['2012'].mean(axis=0))**0.2-1)})
CAGR = pd.DataFrame(CAGR)
CAGR['RegionName']=CAGR.index
CAGR['color'] = CAGR['CAGR'].apply(get_scl,number = 10000)
CAGR_2 = pd.merge(CAGR, center_cord, left_on='RegionName', right_on='county')
CAGR_2["lon"] = ['-'+i[1:9] for i in CAGR_2["lon"]]
CAGR_2["lat"] = [i[1:8] for i in CAGR_2["lat"]]
CAGR_2 = CAGR_2.join(ser, on='RegionName')
layers_ls = []
for x in all_data_sub.index:
    item_dict = dict(sourcetype = 'geojson',
                     source = all_data_sub.ix[x]['coordinates'],
                     type = 'fill',
                     color = all_data_sub.ix[x]['color'])
    layers_ls.append(item_dict)
test = []
for i in range(len(all_data_sub_cen["RegionName"].tolist())):
    test.append(all_data_sub_cen["RegionName"].tolist()[i] + '</br>' + 
                'Sold Price: ' +str(all_data_sub_cen["sold_15"].tolist()[i])+ '</br>' + 
                'List Price: ' +str(all_data_sub_cen["list_15"].tolist()[i])+ '</br>' + 
                'School Score: '+str(all_data_sub_cen["Score2015"].tolist()[i])+ '</br>' +
                'Population:' +str(all_data_sub_cen["Population"].tolist()[i]) +'</br>' +
                'Ave Income: '+str(all_data_sub_cen["income_rate"].tolist()[i]) +'</br>' +
                'Criminal Rate: '+str(all_data_sub_cen["crim_rate"].tolist()[i])+'</br>' +
                'Unemploy Rate: '+str(all_data_sub_cen["Unemploy_Rate"].tolist()[i]))


,CAGR,RegionName
Los Angeles,0.069134,Los Angeles
Orange,0.066662,Orange
San Diego,0.064706,San Diego
Riverside,0.077342,Riverside
San Bernardino,0.085963,San Bernardino


In [93]:
layers_ls = []
for x in all_data_sub.index:
    item_dict = dict(sourcetype = 'geojson',
                     source = CAGR.ix[x]['coordinates'],
                     type = 'fill',
                     color = CAGR.ix[x]['color'])
    layers_ls.append(item_dict)

test = []
for i in range(len(CAGR["RegionName"].tolist())):
    test.append(CAGR["RegionName"].tolist()[i] + '</br>' + 
                'CAGR: ' +str(CAGR["CAGR"].tolist()[i]))
    
data = go.Data([
        go.Scattermapbox(
            lat=CAGR["lat"].tolist(),
            lon=CAGR["lon"].tolist(),
            text=test,
            mode = 'markers',
                    marker = go.Marker(cmax=int(CAGR['CAGR'].max()/0.01),cmin=0,colorscale = colorscl,
                        showscale = True,autocolorscale=False,color=range(0,100),
                    colorbar= go.ColorBar(len = .89)
                                       ),
        )
    ])


layout = go.Layout(
    height=600,
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        layers=layers_ls,
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=39,
            lon=-124
        ),
        pitch=0,
        zoom=5,
        style='light'
               )
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='county-level-choropleths-python')

KeyError: 5